In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("darkgrid")
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import KFold
import itertools as it

In [3]:
names = (['B1', 'B2', 'B3'] + ['C' + str(k) for k in range(1, 140)] + ['YEAR'] +
         ['C140', 'C141', 'C142'] + ['CT' + str(k) for k in range(1, 27)] +
         ['CH' + str(k) for k in range(1, 5)] + ['Class']
        )
feature_names = names[:-1]

In [5]:
dat = pd.read_csv('train.txt',  delimiter='\s+', names=names).dropna()
test_dat = pd.read_csv('prelim-noclass.txt',  delimiter='\s+', names=feature_names).dropna()

In [20]:
dat = dat.drop(['CT7', 'CT8', 'CH2', 'CH4'], axis=1)      ##clean columns that all equal

In [21]:
test_dat = test_dat.drop(['CT7', 'CT8', 'CH2', 'CH4'], axis=1)        ##clean columns that all equal

## Logistic regression

In [22]:
from sklearn.linear_model import LogisticRegression

In [23]:
X = dat.drop(['Class'], axis=1); y = dat['Class']

In [26]:
n_sample = X.shape[0]
cv = ShuffleSplit(n_splits=5, test_size=0.1, random_state=0)
clf = LogisticRegression(max_iter=10000)
cross_val_score(clf, X, y.ravel(), cv=cv)

C:\Users\Dat\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\Dat\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = 

array([0.53583333, 0.52      , 0.52916667, 0.54166667, 0.56916667])

In [29]:
X_train, X_test, y_train, y_test = train_test_split(X, y.ravel(), test_size=0.1, random_state=1)

In [44]:
clf = LogisticRegression(max_iter=50000)
clf.fit(X_train, y_train)

C:\Users\Dat\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of f AND g EVALUATIONS EXCEEDS LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(max_iter=50000)

In [45]:
clf.score(X_train, y_train)

0.6962037037037037

In [46]:
clf.score(X_test, y_test)

0.6766666666666666

## SVM

In [47]:
from sklearn.svm import SVC

In [48]:
clf = SVC()
clf.fit(X_train, y_train)

SVC()

In [49]:
clf.score(X_train, y_train)

0.5375

In [50]:
clf.score(X_test, y_test)

0.5266666666666666

In [51]:
kf = KFold(n_splits=5)

params = {
    'kernel': ['rbf', 'poly', 'sigmoid'],
    'C': [0.1, 1]
 
}

allNames = sorted(params)
combinations = it.product(*(params[Name] for Name in allNames))
combi = list(combinations)
accs = []

for para in combi:
    dict_para = dict(zip(allNames, list(para)))
    dict_acc = dict_para.copy()
    clf = SVC(**dict_para)
    scores = []
    for train_index, test_index in kf.split(X):
        X_train, X_test = X.values[train_index], X.values[test_index]
        y_train, y_test = y.values[train_index], y.values[test_index]
        clf.fit(X_train, y_train)
        score = clf.score(X_test, y_test)
        scores.append(score)
    mean_score = np.mean(scores)
    dict_acc.update({
        'acc': mean_score
    })
    accs.append(dict_acc)

In [52]:
accs = pd.DataFrame(accs)

In [53]:
accs

,C,kernel,acc
0,0.1,rbf,0.521250
1,0.1,poly,0.523000
2,0.1,sigmoid,0.499083
3,1.0,rbf,0.526167
4,1.0,poly,0.520750
5,1.0,sigmoid,0.497750
